In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import html
import json
import time
from requests.exceptions import ConnectionError, TooManyRedirects
import unicodedata
import mysql.connector

In [3]:
con = mysql.connector.connect(user='root', password='', host='127.0.0.1', database='news_db')

### Handling Miss The Content

In [5]:
raw = pd.read_sql("select id, url, source from article where source='propertiterkini' and post_id in (1404243087, 1404232294, 1395334826, 1394033486, 1392825674, 1392310913, 1392139913, 1390881676, 1389808883, 1385488882, 1385134291, 1382581893, 1381465895, 1381465884, 1365300105, 1355324506, 1351344679)", con)
for index, row in raw.iterrows():
    print(row['url'])
    response = requests.get(row['url'])
    html = response.text
    soup = BeautifulSoup(html, "html5lib")
    
    article =  soup.find('div', class_='entry')
           
    for share in article.findAll('div', class_='share-post'):
        if "share" in share.get_text(strip=True).lower():
            share.decompose()

    for image in article.findAll('td'):
        image.decompose()

    for related in article.findAll('section', {'id':'related_posts'}):
        related.decompose()

    for ul in article.findAll('ul'):
        ul.decompose()

    for baca in article.findAll('p'):
        if "baca juga:" in baca.text.lower():
            baca.decompose()

    # for baca2 in article.findAll('div', attrs = {'style':'text-align: justify;'}):
    #     if "baca juga:" in baca2.text.lower():
    #         baca2[:-1].decompose()

    for com in article.findAll('p'):
        if "propertiterkini.com" in com.text.lower():
            baca.decompose()

    for share in article.findAll('div', class_='sharedaddy sd-sharing-enabled'):
        share.decompose()

    for like in article.findAll('h3',class_='sd-title'):
        like.decompose()

    for like2 in article.findAll('span',class_='button'):
        like2.decompose()

    for like3 in article.findAll('div',class_='likes-widget-placeholder post-likes-widget-placeholder'):
        like3.decompose()
        
    detail = BeautifulSoup(article.decode_contents().replace('<br/>', ' '), "html5lib")
    content = re.sub(r'\n|\t|\b|\r','',unicodedata.normalize("NFKD",detail.get_text(strip=True)))
    content = content.replace("‘","").replace("‘","").replace("'","")
                
    cursor = con.cursor()
    sql = "UPDATE article SET content = '"+content+"' WHERE id = "+str(row['id'])
    cursor.execute(sql)
    con.commit()
    cursor.close()


https://www.propertiterkini.com/bisnis-lampu-rotari-yanto-raup-omzet-miliaran-rupiah-1/
https://www.propertiterkini.com/yanto-mantan-anak-jalanan-yang-sukses-dengan-omzet-miliaran-2/
https://www.propertiterkini.com/kisah-insinyur-pesawat-yang-jatuh-bangun-jualan-mebel/
https://www.propertiterkini.com/bermula-suka-inayah-panen-omzet-dari-bisnis-tas-jeans/
https://www.propertiterkini.com/mantan-supir-truk-pun-bisa-jadi-miliarder/
https://www.propertiterkini.com/bunarto-dan-temuannya-untuk-instalasi-gas-medis-rs-yang-efisien-dan-reasonable-2/
https://www.propertiterkini.com/bunarto-hardjasaputra-intens-edukasi-rs-akan-pentingnya-produk-lokal-gas-medis-1/
https://www.propertiterkini.com/arief-widhiyasa-drop-out-demi-game-kini-ceo/
https://www.propertiterkini.com/berdayakan-kreativitas-masyarakat-lokal-apa-adanya/
https://www.propertiterkini.com/peluang-usaha-nanik-mendulang-rupiah-dari-rajutan/
https://www.propertiterkini.com/kisah-hironimus-pala-yang-sukses-berdayakan-petani-kampung/
http